<a href="https://colab.research.google.com/github/callor/Callor-AI-openLama-2025/blob/master/Llama_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset
import torch
from huggingface_hub import notebook_login

# Authenticate with Hugging Face
notebook_login()

# 모델과 토크나이저 로드
base_model = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    device_map="auto",
)

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
401 Client Error. (Request ID: Root=1-67ea2d62-321ba64e43525de77c44b8c1;86019fc7-355b-4185-ac08-a89e394c4e05)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must have access to it and be authenticated to access it. Please log in.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset
import torch

In [4]:
# 모델과 토크나이저 로드
base_model = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    device_map="auto",
)

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
401 Client Error. (Request ID: Root=1-67ea2d08-27a9f95057cc82914da73a27;8bd101be-ab14-4395-b6b8-b62f3a4374ed)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# 예시용 데이터셋
# 실제로는 instruction-output 형태로 준비된 JSONL 파일을 Dataset으로 로드해야 함
example_data = {
    "instruction": ["Java로 파일을 읽는 코드를 작성해줘."],
    "output": [
        "import java.io.*;\npublic class ReadFile {\n public static void main(String[] args) throws IOException {\n BufferedReader reader = new BufferedReader(new FileReader(\"input.txt\"));\n String line;\n while ((line = reader.readLine()) != null) {\n System.out.println(line);\n }\n reader.close();\n }\n}"
    ]
}

In [ ]:
def format_prompt(example):
    return f"<s>[INST] {example['instruction']} [/INST] {example['output']} </s>"

# 데이터 전처리
raw_dataset = Dataset.from_dict(example_data)
raw_dataset = raw_dataset.map(lambda x: {"text": format_prompt(x)})

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./llama3-java-lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False,
)

# 데이터 로더용 콜레이터
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# 저장
trainer.save_model("./llama3-java-lora")
tokenizer.save_pretrained("./llama3-java-lora")